### 문제 2-3 : 학생 정보 구조화 시스템

In [41]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_T


In [42]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint
import json

In [45]:
# 출력 구조를 정의하는 Pydantic 모델
class StudentInformationExtractor(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="목표")

    # Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInformationExtractor)

In [46]:
#프롬프트 템플릿 설정
template = """
다음 학생의 자유 형식 자기소개에서 이름, 나이, 전공, 취미 리스트, 목표를 json 형식으로 추출해 주세요.
학생의 자유 형식 자기소개: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [47]:
#ChatOpenAI 모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [48]:
# 체인 구성 및 실행
query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩하기가 있습니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다!!"
chain = prompt | model | parser
output = chain.invoke({"query": query})

In [49]:
#Pydantic 모델 객체를 딕셔너리로 변환
output_dict = output.model_dump()

#딕셔너리를 json 형식의 문자열로 변환
json_output = json.dumps(output_dict, indent=4, ensure_ascii=False)

print(json_output)

{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩하기"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}
